In [ ]:
def train_test_split(df):
    from sklearn.model_selection import train_test_split
    print("The datset for training is:")
    print(df.shape)
    df.info()
    df.head(10)    

    Independent_var = df.lemmatized_review
    Dependent_var = df.Sentiment

    IV_train, IV_test, DV_train, DV_test = train_test_split(Independent_var, Dependent_var, test_size = 0.2, random_state = 225)

    print('IV_train :', len(IV_train))
    print('IV_test  :', len(IV_test))
    print('DV_train :', len(DV_train))
    print('DV_test  :', len(DV_test))

    IV_train.to_csv(f'../data/interim/X_train.csv', index=False)
    IV_test.to_csv(f'../data/interim/X_test.csv', index=False)
    DV_train.to_csv(f'../data/interim/y_train.csv', index=False)
    DV_test.to_csv(f'../data/interim/y_test.csv', index=False)
def training_classifier():
    import pandas as pd
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression

    tvec = TfidfVectorizer()
    clf2 = LogisticRegression(solver = "lbfgs")


    from sklearn.pipeline import Pipeline
    IV_train= pd.read_csv('../data/interim/X_train.csv', index_col=0, dtype=str)
    DV_train= pd.read_csv('../data/interim/y_train.csv', index_col=0, dtype=str)
    
    model = Pipeline([('vectorizer',tvec),('classifier',clf2)])

    model.fit(IV_train, DV_train)

    import pickle
    with open(f'../data/external/model.pkl', 'wb') as f:
        pickle.dump(model, f)

    print("\n logistic regression classifier is trained on sentiment data and saved to PV location ../external/model.pkl ----")
def predict_test_data():
    import pandas as pd
    import numpy as np
    import pickle
    from sklearn.metrics import confusion_matrix
    
    print("----Inside predict_test_data stage ----")
    with open(f'../data/external/model.pkl', 'rb') as f:
        model = pickle.load(f)

    IV_test= pd.read_csv('../data/interim/X_test.csv', index_col=0)
    DV_test= pd.read_csv('../data/interim/y_test.csv', index_col=0)
    predictions = model.predict(IV_test)
    np.save(f'../data/external/y_pred.npy', predictions)

    confusion_matrix(predictions, DV_test)
def get_metrics():
    import pandas as pd
    import numpy as np
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    
    predictions= pd.read_csv('./data/external/y_pred.npy')
    DV_test= pd.read_csv('../data/interim/y_test.csv')

    print("Accuracy : ", accuracy_score(predictions, DV_test))
    print("Precision : ", precision_score(predictions, DV_test, average = 'weighted'))
    print("Recall : ", recall_score(predictions, DV_test, average = 'weighted'))